# San Francisco Parking Exploratory Data Analysis
this notebook explores a few extra features I added to the project. This notebook will contain 
-Live graph of tickets over a single day
-Plotting Recently cleaned Streets
-Estimated Sweeping Time

The scripts also add on this, plotting and making videos of the street sweepers routes, and plotting the estimated model output. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib 
import numpy as np
import datetime as dt
import time
from scipy import stats
import geopandas as gpd
import sqlite3
import math
from shapely.geometry import Point
from geopandas import GeoSeries, GeoDataFrame
import mplleaflet

%matplotlib inline

project_folder = '/home/colin/Desktop/SF_Parking/'
raw_folder = project_folder + 'data/raw/'
proc_folder = project_folder + 'data/processed/'
image_folder = project_folder + 'reports/figures/analysis/'

def result_query(querystring):
    resultdf = pd.read_sql(sql= querystring, con = conn)
    
    return resultdf

conn = sqlite3.connect(proc_folder + 'SF_Parking.db')
c = conn.cursor()



# Live Graph
We can use the matplotlib animation to show where tickets happened throughout the day given any specific day in time. I just think this one would be cool, no particular help or exploratory value. Because we would like to separate ticket type by colors, all else other than the top 5 will be one color. 

In [ ]:
by_day = result_query("Select  strftime('%d-%m-%Y', TickIssueDate), count(*) as total_tickets "
                      "from ticket_data group by  strftime('%d-%m-%Y', TickIssueDate) order by count(*) desc")
by_day.head()

In [ ]:
#Make a color Dictionary for each ticket Type
colordict = {'STR CLEAN': 'cyan', 'RES/OT': 'green', 'MTR OUT DT': 'red', 'DRIVEWAY': 'orange', 'DBL PARK':'blue'}
df = result_query("Select * from ticket_data where strftime('%d-%m-%Y', TickIssueDate) = '05-09-2017'")

In [ ]:
df

In [ ]:
address_data = result_query('Select * from address_data')
df = df.merge(address_data, left_on = 'address', right_on = 'address')
                                 

In [ ]:
df['color'] = df['ViolationDesc'].apply(lambda x: colordict.get(x, 'magenta'))
df

In [ ]:
streetvolume = gpd.read_file(proc_folder + './final_streets/SF_Street_Data.shp')
geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
nhoods = gpd.read_file(raw_folder + 'AnalysisNeighborhoods.geojson')

base = streetvolume.plot(color = 'black', figsize = (20, 20), alpha =.25, linewidth = 1)
for key, value in colordict.items():
    gdf[gdf.ViolationDesc == key].plot(ax = base, marker = "*", color=value, markersize=5);
nhoods.plot(ax = base, alpha = .15, color = 'gray')


In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
fig = plt.figure(figsize = (20,20))
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 100
ax = plt.axes()
df = result_query("Select * from ticket_data where strftime('%d-%m-%Y', TickIssueDate) = '05-09-2017'")
address_data = result_query('Select * from address_data')
df = df.merge(address_data, left_on = 'address', right_on = 'address')
df['color'] = df['ViolationDesc'].apply(lambda x: colordict.get(x, 'magenta'))
ax.set_title('San Francisco Parking tickets on 05-09-2017', fontdict = {'fontsize' : 20})
geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
nhoods = gpd.read_file(raw_folder + 'AnalysisNeighborhoods.geojson')
plt.axis('off')
plt.tight_layout()


nhoods.plot( ax = ax, alpha = .15, color = 'gray')
# First set up the figure, the axis, and the plot element we want to animate
streetvolume.plot(ax =ax, color = 'black', figsize = (20, 20), alpha =.25, linewidth = 1)
gdf.sort_values(by = 'TickIssueDate', inplace = True)
gdf['TickIssueDate'] = gdf['TickIssueDate'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
gdf['TickIssueTime'] = gdf['TickIssueDate'].apply(lambda x: x.time().hour*4 + int(x.time().minute / 15))
gdf.set_index('TickIssueTime', inplace = True)
ttl = ax.text(.5, 1.05, '', transform = ax.transAxes, va='center')
numframes = gdf.shape[0]
i = 0

def animate(i):
    df = gdf[i-1:i]
    timestr = (str(math.floor(i/4)) + ':' + str((i %4) * 15))
    colors = df['color']
    iterar = df.plot(ax = ax, marker = '*', c = colors, markersize = 10 )
    ttl.set_text(timestr)
    i += 1
    return iterar

ani = FuncAnimation(fig, animate, repeat=False, interval=94)
#plt.show()
HTML(ani.to_jshtml())



# Recent Street Cleaning
This will take an address as an argument, and return the closest streets that were sweeped that day. These are typically easier to park in. Optional argument for streets that have never gotten a 'Residential Over Time Ticket'

In [ ]:
streetsweeping = gpd.read_file(proc_folder + 'final_sweeping.shp')
weekdaydict = {0: 'Mon', 1:'Tues', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
time =  dt.datetime.now()
cleaned_today = streetsweeping[streetsweeping.weekday == weekdaydict[time.weekday()]]
cleaned_today.plot(figsize = (20,20))
plt.title('Recently Cleaned Streets')
#Lets make a helper function as a bonus to identify recently cleaned streets that would be easier to park at

In [ ]:
from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

In [ ]:
streetvolume.head()

In [ ]:
streetvolume.dtypes

In [ ]:
streetvolume = gpd.read_file(proc_folder + 'final_streets/SF_Street_Data.shp')
streetvolume = streetvolume[['vvol_carea', 'geometry']]
streetvolume = streetvolume.to_crs(epsg = 4326)
streetvolume.plot()

In [ ]:
streetindex = streetvolume.index.values.astype(str)
streetvolume.set_index(streetindex, inplace = True)

In [ ]:
da = streetvolume.index.values

In [ ]:
da

In [ ]:
streetvolume

In [ ]:
base = streetvolume.plot( column = 'vvol_carea', cmap = 'RdYlGn', alpha = .75)
mplleaflet.show(fig=base.figure, crs=streetvolume.crs, tiles='cartodb_positron')

In [ ]:


def find_recent_street_cleaning(streetnumber, streetname, ResOT =False):
    ad = result_query("Select * from address_data where street = '" + streetname + "'" )
    if ad.shape[0] == 0:
        return print('Could not find streetname')
    ad['delta'] = np.abs(ad['number'] - streetnumber)
    ad.sort_values(by = 'delta', inplace = True)
    ad = ad.iloc[0]
    df = streetsweeping
    point = Point(ad.lon, ad.lat)
    if ResOT == True:
        invalid_ids = result_query('Select distinct lineid from address_data t1 join ticket_data t2 on '
                                  " t1.address = t2.address where ViolationDesc = 'RES/OT' ")
        
        df = df[df.lineid.isin(invalid_ids['lineid']) == False]
        
    weekdaydict = {0: 'Mon', 1:'Tues', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    time =  dt.datetime.now()
    
    colname = 'week' + str(week_of_month(time)) + 'ofmon'
    cleaned_today = df[(df.weekday == weekdaydict[time.weekday()]) & (df[colname] == 1)]
    not_today = df[(df.weekday != weekdaydict[time.weekday()]) | (df[colname] == 0)]
    nhoods = gpd.read_file(raw_folder + '/AnalysisNeighborhoods.geojson')
    cleaned_today['distance'] = cleaned_today['geometry'].apply(lambda x: point.distance(x))
    cleaned_today.sort_values(by = 'distance', inplace = True)
    cleaned_today_closest = cleaned_today[:25]
    


    #ax = not_today.plot(color = 'red', alpha = .15)
    ax = cleaned_today.plot( color = 'yellow', alpha = .75)
    cleaned_today[:500].plot(ax = ax, color = 'green', alpha = 1)
    streetvolume.plot(ax = ax, column = 'vvol_carea', cmap = 'RdYlGn')
    circleaddress = matplotlib.patches.Circle((ad['lon'], ad['lat']), radius = 5)
    mplleaflet.show(fig=ax.figure, crs=cleaned_today.crs, tiles='cartodb_positron')
    return ax
                              

In [ ]:
p = find_recent_street_cleaning(980, 'BUSH ST', ResOT =False
                            )
plt.show()

# Estimated Sweeping Time
Finding a spot is a tough gig in San Francisco, so I've found the best practice is to know when the sweeping truck is coming by and try to follow him. Unsurprisingly, I'm not the only one who does this and it usually results in a death match between all cars. If you put in an address here, I'll give you an estimate as well a confidence interval on the range of times I would expect the truck to arrive, so you don't have to sit around and wait. 

In [ ]:
by_route = result_query("Select  strftime('%Y-%m-%d', TickIssueDate) as sweepdate, lineid, "
                        " max(strftime('%H:%M',TickIssueDate)) as last_ticket from ticket_data t1 "
                       " join address_data t2 on t1.address = t2.address WHERE ViolationDesc = 'STR CLEAN' "
                    " group by strftime('%Y-%m-%d', TickIssueDate) ,  lineid")

In [ ]:
by_route['weekday'] = by_route['sweepdate'].apply(lambda x: pd.to_datetime(x).weekday())
by_route['mins'] = by_route['last_ticket'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
def min_to_time(mins):
    return str(math.ceil(mins / 60)) + ":" + str(int(mins%60))

In [ ]:
def return_conf_interval(number, street):
    ad = result_query("Select * from address_data where street = '" + street + "'" )
    if ad.shape[0] == 0:
        return print('Could not find streetname')
    ad['delta'] = np.abs(ad['number'] - number)
    ad.sort_values(by = 'delta', inplace = True)
    streeline = ad['lineid'].iloc[0]
    street_data = by_route[by_route.lineid == streeline]
    if street_data.shape[0] == 0:
        return print('No street sweeping ticket data found for closest address. ')
    mean, low, high = mean_confidence_interval(street_data['mins'])
    print("low: " + min_to_time(low))
    print("mean: " + min_to_time(mean))
    print("High: " + min_to_time(high))

In [ ]:
return_conf_interval(980, 'BUSH ST')